# Elastic Modulus Comparison

This notebook compares three different methods for calculating elastic modulus of snow layers:
1. **Bergfeld et al. (2023)** - Power-law based on density
2. **Köchle and Schneebeli (2014)** - Exponential relationships from μ-CT and FE simulations  
3. **Wautier et al. (2015)** - Power-law with ice modulus parameter

All three methods require both **density** and **grain form** as inputs.


In [13]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from uncertainties import ufloat
import warnings


# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


In [15]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Kim_geldsetzer densities

In [16]:
# Collect relevant data from each snowpit
layer_info = []

for pit in all_pits:
    for layer in pit.snow_profile.layers:
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer.depth_top[0] if layer.depth_top else None,  
            'thickness': layer.thickness[0] if layer.thickness else None,
        }

        # Convert grain form and calculate density
        grain_form = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer') if layer.grain_form_primary else None
        layer_dict['kim_geldsetzer_grain_form'] = grain_form
        
        # Calculate density if we have valid inputs
        if layer.hardness and grain_form:
            try:
                density_ufloat = density.calculate_density( 
                    method='kim_geldsetzer',
                    hand_hardness=layer.hardness,
                    grain_form=grain_form
                )
                layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
                layer_dict['density_kim_geldsetzer_uncertainty'] = None
        else:
            layer_dict['density_kim_geldsetzer'] = None
            layer_dict['density_kim_geldsetzer_uncertainty'] = None

        layer_info.append(layer_dict)

# Create dataframes
layer_df = pd.DataFrame(layer_info)


In [18]:
# Implement the elastic modulus calculations

def calculate_emod_row(row):
    """Calculate elastic modulus for a single row using multiple methods."""
    result = {
        'e_mod_bergfeld': np.nan,
        'e_mod_kochle': np.nan,
        'e_mod_wautier': np.nan
    }
    
    # Skip if no grain form or density data
    if pd.isna(row['kim_geldsetzer_grain_form']) or pd.isna(row['density_kim_geldsetzer']):
        return pd.Series(result)
    
    try:
        # Create density ufloat once
        density_ufloat = ufloat(row['density_kim_geldsetzer'], row['density_kim_geldsetzer_uncertainty'])
        grain_form = row['kim_geldsetzer_grain_form']
        
        # Calculate elastic modulus using Bergfeld method (needs grain_form!)
        try:
            E_mod_bergfeld = elastic_modulus.calculate_elastic_modulus(
                method='bergfeld',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_bergfeld'] = E_mod_bergfeld.nominal_value
        except Exception:
            pass  # Keep as np.nan
        
        # Calculate elastic modulus using Kochle method
        try:
            E_mod_kochle = elastic_modulus.calculate_elastic_modulus(
                method='kochle',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_kochle'] = E_mod_kochle.nominal_value
        except Exception:
            pass  # Keep as np.nan
        
        # Calculate elastic modulus using Wautier method
        try:
            E_mod_wautier = elastic_modulus.calculate_elastic_modulus(
                method='wautier',
                density=density_ufloat,
                grain_form=grain_form
            )
            result['e_mod_wautier'] = E_mod_wautier.nominal_value
        except Exception:
            pass  # Keep as np.nan
        
    except Exception:
        pass  # result already contains np.nan values
    
    return pd.Series(result)

# Apply the function to all rows at once
layer_df[['e_mod_bergfeld', 'e_mod_kochle', 'e_mod_wautier']] = layer_df.apply(calculate_emod_row, axis=1)


/Users/marykateconnelly/Desktop/F25/SnowPyt-MechParams/venv/lib/python3.13/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [ ]:
# Check the results
print("Elastic Modulus Calculation Summary:")
print("=" * 60)
print(f"\nTotal layers: {len(layer_df)}")
print(f"Layers with density data: {layer_df['density_kim_geldsetzer'].notna().sum()}")
print(f"Layers with grain form data: {layer_df['kim_geldsetzer_grain_form'].notna().sum()}")
print(f"\nSuccessful calculations:")
print(f"  Bergfeld: {layer_df['e_mod_bergfeld'].notna().sum()}")
print(f"  Kochle:   {layer_df['e_mod_kochle'].notna().sum()}")
print(f"  Wautier:  {layer_df['e_mod_wautier'].notna().sum()}")

# Show a sample of successful calculations
successful = layer_df[layer_df['e_mod_bergfeld'].notna() | 
                      layer_df['e_mod_kochle'].notna() | 
                      layer_df['e_mod_wautier'].notna()]


Elastic Modulus Calculation Summary:

Total layers: 371429
Layers with density data: 235522
Layers with grain form data: 259731

Successful calculations:
  Bergfeld: 88319
  Kochle:   101079
  Wautier:  203401


Sample of successful calculations (first 5):
   pit_id hand_hardness kim_geldsetzer_grain_form  density_kim_geldsetzer  \
3   51686            F+                        DF                 112.242   
5   51686            F+                      FCxr                 146.738   
7    5847            4F                        DF                 137.300   
9    5847             F                      FCxr                 127.400   
12   9544             P                      MFcr                 295.400   

    e_mod_bergfeld  e_mod_kochle  e_mod_wautier  
3         0.629756           NaN       6.064847  
5              NaN           NaN      11.354445  
7         1.528406           NaN       9.718635  
9              NaN           NaN       8.157417  
12             NaN    155.8271

In [20]:
#layer_df.to_csv('emod_layer_df.csv')
